In [ ]:
import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import graphviz 
from sklearn import model_selection
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, learning_curve, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score 
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier as MLPC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
data = read_csv('E:/中科大/2024年工作进程/ddrtree2/ahsl0519.csv')
X = data.iloc[:, 1:]  # 选择除了第一列以外的所有列作为特征变量 X
y = data.iloc[:, 0]   # 选择第一列作为目标变量 y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train3 = pd.DataFrame(X_train)
print(y_train.dtype)  # 输出 y_train 的数据类型
print(y_train.unique())  # 输出 y_train 中的唯一值

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier  # 导入 GBC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
import torch.nn as nn
import torch.optim as optim

# set seed
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

# load data
data = pd.read_csv('.csv')

# feature and label
features = data.drop(columns=['diabetes'])
labels = data['diabetes']

# Standardized  data
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# trans PyTorch
features_scaled = torch.tensor(features_scaled, dtype=torch.float32)
labels = torch.tensor(labels.values, dtype=torch.float32)

# define VAE
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU()
        )
        self.mean_linear = nn.Linear(64, latent_dim)
        self.var_linear = nn.Linear(64, latent_dim)
        self.decoder_projection = nn.Linear(latent_dim, 64)
        self.decoder = nn.Sequential(
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        mean = self.mean_linear(encoded)
        logvar = self.var_linear(encoded)
        eps = torch.randn_like(logvar)
        std = torch.exp(logvar / 2)
        z = eps * std + mean
        x = self.decoder_projection(z)
        decoded = self.decoder(x)
        return decoded, mean, logvar
    
    def find_key_features(self):
        weights = self.encoder[0].weight.data.numpy()
        feature_importance = np.sum(np.abs(weights), axis=0)
        return feature_importance

input_dim = features_scaled.shape[1]
latent_dim = 10
vae = VAE(input_dim, latent_dim)

# define loss
def loss_function(recon_x, x, mean, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    return BCE + KLD

# 
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# train VAE
epochs = 50
batch_size = 32
for epoch in range(epochs):
    vae.train()
    train_loss = 0
    for i in range(0, len(features_scaled), batch_size):
        batch_features = features_scaled[i:i+batch_size]
        optimizer.zero_grad()
        recon_batch, mean, logvar = vae(batch_features)
        loss = loss_function(recon_batch, batch_features, mean, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {train_loss / len(features_scaled)}')

# extract VAE 
vae.eval()
with torch.no_grad():
    encoded_features = vae.encoder(features_scaled)
    mean = vae.mean_linear(encoded_features)

# trans numpy
encoded_features = mean.numpy()

labels = labels.numpy()

# cut data
X_train, X_test, y_train, y_test = train_test_split(encoded_features, labels, test_size=0.2, random_state=42)

# GBC
clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# predict
y_pred = clf.predict(X_test)

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

feature_importance = clf.feature_importances_
feature_names = features.columns

# TOP10
top_10_indices = np.argsort(feature_importance)[-10:][::-1]
top_10_features = feature_names[top_10_indices]
top_10_values = feature_importance[top_10_indices]

# plot
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_values, y=top_10_features, palette='viridis')
plt.title('Top 10 Features')
plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
plt.show()
